<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/03_xyz_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-19 20:59:59--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12074 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.79K  --.-KB/s    in 0s      

2022-06-19 20:59:59 (117 MB/s) - ‘utils.py’ saved [12074/12074]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [1]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import ModelCreator, get_train_val_ds, get_class_weight

In [2]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [3]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [4]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3510 files belonging to 16 classes.


In [5]:
classes, class_weight = get_class_weight()

In [6]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [7]:
xyz_creator = ModelCreator(HUB_URL, 'XYZ-model')
xyz_model = xyz_creator.make_model(img_size=IMG_SIZE)

Model: "XYZ-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23500352  
                                                                 
 dense (Dense)               (None, 16)                32784     
                                                                 
Total params: 23,533,136
Trainable params: 32,784
Non-trainable params: 23,500,352
_________________________________________________________________


In [8]:
callbacks = xyz_creator.get_callbacks()

In [9]:
xyz_model.layers[0].trainable = True
xyz_creator.re_compile(xyz_model, lr=1e-3)

In [10]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=5,
                        class_weight=class_weight,
                        validation_data=val_ds)

Epoch 1/5
519/519 [==============================] - 647s 1s/step - loss: 43.6819 - auc: 0.0992 - accuracy: 0.1245 - val_loss: 2.3662 - val_auc: 0.2898 - val_accuracy: 0.3980
Epoch 2/5
519/519 [==============================] - 416s 800ms/step - loss: 34.2436 - auc: 0.2298 - accuracy: 0.2380 - val_loss: 2.1564 - val_auc: 0.3088 - val_accuracy: 0.3151
Epoch 3/5
519/519 [==============================] - 421s 808ms/step - loss: 26.5571 - auc: 0.4196 - accuracy: 0.3909 - val_loss: 1.7387 - val_auc: 0.4589 - val_accuracy: 0.4157
Epoch 4/5
519/519 [==============================] - 421s 809ms/step - loss: 21.1374 - auc: 0.5844 - accuracy: 0.5197 - val_loss: 1.6311 - val_auc: 0.5204 - val_accuracy: 0.4917
Epoch 5/5
519/519 [==============================] - 422s 809ms/step - loss: 18.2735 - auc: 0.6727 - accuracy: 0.5957 - val_loss: 1.3664 - val_auc: 0.6487 - val_accuracy: 0.5855


In [11]:
!cp ./logs/checkpoints/XYZ-model-05.h5 ./drive/MyDrive/checkpoints/xyz-model-ft-5.h5

In [ ]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=10,
                        initial_epoch=5,
                        class_weight=class_weight)

Epoch 21/30
519/519 [==============================] - 179s 339ms/step - loss: 3.6200 - auc: 0.9587 - accuracy: 0.9388
Epoch 22/30
519/519 [==============================] - 180s 344ms/step - loss: 3.2040 - auc: 0.9687 - accuracy: 0.9495
Epoch 23/30
519/519 [==============================] - 178s 340ms/step - loss: 3.0689 - auc: 0.9666 - accuracy: 0.9493
Epoch 24/30
519/519 [==============================] - 180s 344ms/step - loss: 2.4269 - auc: 0.9754 - accuracy: 0.9572
Epoch 25/30
519/519 [==============================] - 183s 349ms/step - loss: 2.1273 - auc: 0.9786 - accuracy: 0.9602
Epoch 26/30
519/519 [==============================] - 196s 374ms/step - loss: 1.9823 - auc: 0.9801 - accuracy: 0.9608
Epoch 27/30
519/519 [==============================] - 178s 340ms/step - loss: 1.7686 - auc: 0.9837 - accuracy: 0.9660
Epoch 28/30
519/519 [==============================] - 177s 338ms/step - loss: 1.6524 - auc: 0.9842 - accuracy: 0.9668
Epoch 29/30
519/519 [===========================

In [ ]:
!cp ./logs/checkpoints/XYZ-model-25.h5 ./drive/MyDrive/checkpoints/xyz-model-25.h5
!cp ./logs/checkpoints/XYZ-model-30.h5 ./drive/MyDrive/checkpoints/xyz-model-30.h5

In [ ]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=35,
                        initial_epoch=30,
                        class_weight=class_weight)

Epoch 31/35
519/519 [==============================] - 179s 343ms/step - loss: 1.1576 - auc: 0.9896 - accuracy: 0.9742
Epoch 32/35
519/519 [==============================] - 178s 340ms/step - loss: 1.0805 - auc: 0.9905 - accuracy: 0.9754
Epoch 33/35
519/519 [==============================] - 177s 337ms/step - loss: 1.0385 - auc: 0.9907 - accuracy: 0.9768
Epoch 34/35
519/519 [==============================] - 179s 343ms/step - loss: 0.9734 - auc: 0.9920 - accuracy: 0.9790
Epoch 35/35
519/519 [==============================] - 177s 338ms/step - loss: 0.8809 - auc: 0.9931 - accuracy: 0.9804


In [ ]:
!cp ./logs/checkpoints/XYZ-model-35.h5 ./drive/MyDrive/checkpoints/xyz-model-35.h5